Index and test of serach using BM25

In [1]:
!pip install python-terrier
import pyterrier as pt
import pandas as pd
pt.init()

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.2 MB/s eta 0:00:00
  Created wheel for chest: filename=chest-0.2.3-py3-none-any.whl size=7611 sha256=66623ab77a04195a1b31a489a735257068124fe572

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
<ipython-input-1-df7663096205>:4: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [45]:
!rm -rf ./index

# Make sure this dataset exist before running
df = pd.read_csv("cleaned_dataset.csv")

# Make sure the dataset contains columns of 'docno' and 'text'
if 'docno' not in df.columns or 'text' not in df.columns:
    raise ValueError("CSV file must include columns of 'docno' and 'text' ")

# Transfer dataframe to dictionary form
required_columns = ['docno', 'title', 'text']
doc_iter = df[required_columns].to_dict(orient="records")

# Create Indexer
indexer = pt.IterDictIndexer("./index", meta={'docno': 20, 'title': 512, 'text': 4096}, text_attrs=['title', 'text'],fields=True)

# Genrate Index
indexref = indexer.index(doc_iter)

# load index and print it
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3009
Number of terms: 12984
Number of postings: 173368
Number of fields: 2
Number of tokens: 273340
Field names: [title, text]
Positions:   false



In [21]:
for i, doc in enumerate(doc_iter[:10]):  # 看前10条
    if 'title' not in doc:
        print(f'Missing title in doc {i}:', doc)

In [41]:
doc_iter[0]

{'docno': '"3809"',
 'title': 'asian exporters fear damage u.s.-japan rift',
 'text': "mounting trade friction u.s. japan raised fears among many asia 's exporting nations row could inflict far-reaching economic damage , businessmen officials said . told reuter correspondents asian capitals u.s. move japan might boost protectionist sentiment u.s. lead curbs american imports products . exporters said conflict would hurt long-run , short-term tokyo 's loss might gain . u.s. said impose 300 mln dlrs tariffs imports japanese electronics goods april 17 , retaliation japan 's alleged failure stick pact sell semiconductors world markets cost . unofficial japanese estimates put impact tariffs 10 billion dlrs spokesmen major electronics firms said would virtually halt exports products hit new taxes . `` would n't able business , '' said spokesman leading japanese electronics firm matsushita electric industrial co ltd & lt ; mc.t > . `` tariffs remain place length time beyond months mean complet

A simple test run of BM25(Search US president)

In [42]:
result_df1 = pt.BatchRetrieve(indexref, wmodel="BM25").search("US president")
result_df1

<ipython-input-42-7289b9a150d9>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  result_df1 = pt.BatchRetrieve(indexref, wmodel="BM25").search("US president")


,qid,docid,docno,rank,score,query
0,1,514,"""5529""",0,6.652595,US president
1,1,1372,"""5889""",1,6.578187,US president
2,1,1867,"""6735""",2,6.246572,US president
3,1,2056,"""7256""",3,6.110357,US president
4,1,2045,"""7222""",4,6.066262,US president
...,...,...,...,...,...,...
198,1,1509,"""6209""",198,1.644375,US president
199,1,2605,"""21359""",199,1.628448,US president
200,1,1847,"""6601""",200,1.474232,US president
201,1,2976,"""19928""",201,1.421867,US president


In [44]:
result_df2 = pt.BatchRetrieve(indexref, wmodel="BM25F", properties = {'w.0': 0.5, 'w.1': 1}).search("US president")
result_df2

<ipython-input-44-40c9a7ec8476>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  result_df2 = pt.BatchRetrieve(indexref, wmodel="BM25F", properties = {'w.0': 0.5, 'w.1': 1}).search("US president")


,qid,docid,docno,rank,score,query
0,1,514,"""5529""",0,7.025821,US president
1,1,1867,"""6735""",1,6.820663,US president
2,1,2434,"""20972""",2,6.781061,US president
3,1,2045,"""7222""",3,6.589754,US president
4,1,2056,"""7256""",4,6.589754,US president
...,...,...,...,...,...,...
198,1,1509,"""6209""",198,1.314864,US president
199,1,2605,"""21359""",199,1.300225,US president
200,1,1847,"""6601""",200,1.160670,US president
201,1,2976,"""19928""",201,1.114165,US president
